In [52]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from sklearn.linear_model import LinearRegression
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time

In [20]:
# Obtaining RAPTOR projections for this year
tables = pd.read_html('https://projects.fivethirtyeight.com/2022-nba-predictions/bulls/')
table = tables[0]
table.columns = table.columns.droplevel(0)
table = table[['Player', 'Off. +/-', 'Def. +/-']]
table.columns = ['Player', 'Offensive_Rating', 'Defensive_Rating']
table.head()

,Player,Offensive_Rating,Defensive_Rating
0,Zach LaVine,+3.1,-2.1
1,Nikola Vucevic,-0.4,+1.9
2,DeMar DeRozan,+2.1,-1.8
3,Lonzo Ball,+1.0,+1.2
4,Alex Caruso,+0.2,+3.7
